# Create test data

In [ ]:
import librosa
from datasets import load_dataset


common_voice = load_dataset("common_voice", "ja")
audio_data_list = [
    librosa.resample(
        common_voice['train'][i]['audio']['array'], orig_sr=48000, target_sr=16000
    ) for i in range(10)]

# Official Whisper

In [2]:
import whisper

model = whisper.load_model("large")

In [4]:
%%time

for audio_data in audio_data_list:
    result = model.transcribe(
        audio_data,
        verbose=True,
        language='japanese',
        beam_size=5,
        fp16=True,
        without_timestamps=True
    )

[00:00.000 --> 00:30.000] 予想外の事態に電力会社がちょっぴり困惑切りだ
[00:00.000 --> 00:30.000] 町域にあった峰山藩は長岡藩に米100票を送ったことで有名。
[00:00.000 --> 00:30.000] 週末 友達と山に登ります
[00:00.000 --> 00:30.000] 後で図書館へ本を返しに行きます。
[00:00.000 --> 00:30.000] 55歳だって嬉しい時が嬉しいのだ
[00:00.000 --> 00:30.000] 私はパンもご飯も好きです。
[00:00.000 --> 00:30.000] デパートやスーパーで買い物をします
[00:00.000 --> 00:30.000] 用紙に書いてある番号を覚えます。
[00:00.000 --> 00:30.000] 明日 友達と 映画を 見に行きます。
[00:00.000 --> 00:30.000] あの男の人は背が高くて足が長いです。
CPU times: user 26.9 s, sys: 216 ms, total: 27.1 s
Wall time: 13.6 s


# Official Whisper with model.half()

### We will get a little faster and a large memory improvement (12G -> 6G)

In [2]:
import whisper


model = whisper.load_model("large", device="cpu")
_ = model.half()
_ = model.cuda()

# exception without following code
# reason : model.py -> line 31 -> super().forward(x.float()).type(x.dtype)
for m in model.modules():
    if isinstance(m, whisper.model.LayerNorm):
        m.float()

In [4]:
%%time

for audio_data in audio_data_list:
    result = model.transcribe(
        audio_data,
        verbose=True,
        language='japanese',
        beam_size=5,
        fp16=True,
        without_timestamps=True
    )

[00:00.000 --> 00:30.000] 予想外の事態に電力会社がちょっぴり困惑切りだ
[00:00.000 --> 00:30.000] 町域にあった峰山藩は長岡藩に米100票を送ったことで有名。
[00:00.000 --> 00:30.000] 週末 友達と山に登ります
[00:00.000 --> 00:30.000] 後で図書館へ本を返しに行きます。
[00:00.000 --> 00:30.000] 55歳だって嬉しい時が嬉しいのだ
[00:00.000 --> 00:30.000] 私はパンもご飯も好きです。
[00:00.000 --> 00:30.000] デパートやスーパーで買い物をします
[00:00.000 --> 00:30.000] 用紙に書いてある番号を覚えます。
[00:00.000 --> 00:30.000] 明日 友達と 映画を 見に行きます。
[00:00.000 --> 00:30.000] あの男の人は背が高くて足が長いです。
CPU times: user 25.1 s, sys: 130 ms, total: 25.3 s
Wall time: 12.5 s


# Whisper with TorchScript

In [2]:
import torch
import efficient_whisper as whisper

In [3]:
model = whisper.load_model("large", device="cpu")
model.encoder = torch.jit.script(model.encoder)
model.decoder = torch.jit.script(model.decoder)
_ = model.half()
_ = model.cuda()

In [5]:
%%time

for audio_data in audio_data_list:
    result = model.transcribe(
        audio_data,
        verbose=True,
        language='japanese',
        beam_size=5,
        fp16=True,
        without_timestamps=True
    )

[00:00.000 --> 00:30.000] 予想外の事態に電力会社がちょっぴり困惑切りだ
[00:00.000 --> 00:30.000] 町域にあった峰山藩は長岡藩に米100票を送ったことで有名。
[00:00.000 --> 00:30.000] 週末 友達と山に登ります
[00:00.000 --> 00:30.000] 後で図書館へ本を返しに行きます。
[00:00.000 --> 00:30.000] 55歳だって嬉しい時が嬉しいのだ
[00:00.000 --> 00:30.000] 私はパンもご飯も好きです。
[00:00.000 --> 00:30.000] デパートやスーパーで買い物をします
[00:00.000 --> 00:30.000] 用紙に書いてある番号を覚えます。
[00:00.000 --> 00:30.000] 明日 友達と 映画を 見に行きます。
[00:00.000 --> 00:30.000] あの男の人は背が高くて足が長いです。
CPU times: user 20.8 s, sys: 425 ms, total: 21.2 s
Wall time: 8.59 s


# Whisper with TorchScript & pad_or_trim (30s -> 10s)

Fix `CHUNK_LENGTH` in audio.py: `30 -> 10`

In [2]:
import torch
import efficient_whisper as whisper

In [3]:
checkpoint = torch.load('/home/ubuntu/.cache/whisper/large.pt', map_location='cpu')
dims = whisper.model.ModelDimensions(**checkpoint["dims"])
dims.n_audio_ctx = 500  # 10s

model = whisper.model.Whisper(dims)
for k, p in model.state_dict().items():
    p.copy_(checkpoint["model_state_dict"][k])

model.encoder = torch.jit.script(model.encoder)
model.decoder = torch.jit.script(model.decoder)
_ = model.half()
_ = model.cuda()

In [5]:
%%time

for audio_data in audio_data_list:
    result = model.transcribe(
        audio_data,
        verbose=True,
        language='japanese',
        beam_size=5,
        fp16=True,
        without_timestamps=True
    )

[00:00.000 --> 00:10.000] 予想外の事態に電力会社がちょっぴり手を巻く気味が
[00:00.000 --> 00:10.000] 町域にあった峰山藩は、長岡藩に米100票を送ったことで有名。
[00:00.000 --> 00:10.000] 週末友達と山に登ります。
[00:00.000 --> 00:10.000] 後で図書館へ本を返しに行きます。
[00:00.000 --> 00:10.000] 55歳だって嬉しい時が嬉しいのだ
[00:00.000 --> 00:10.000] 私はパンもご飯も好きです。
[00:00.000 --> 00:10.000] デパートやスーパーで買い物をします。
[00:00.000 --> 00:10.000] 用紙に書いてある番号を覚えます。
[00:00.000 --> 00:10.000] 明日、友達と映画を見に行きます。
[00:00.000 --> 00:10.000] あの男の人は背が高くて足が長いです。
CPU times: user 20.3 s, sys: 3.61 ms, total: 20.3 s
Wall time: 6.77 s
